In [4]:
def create_db():
    try:
        # подключаем базу данных
        conn = sqlite3.connect('meeting_reminder.db')

        # курсор для работы с таблицами
        cursor = conn.cursor()

        # try:
        #     query = 'DROP TABLE "meeting_reminder"'
        #     cursor.execute(query)
        # except:
        #     pass
        #     raise

        try:
            # sql запрос для создания таблицы
            query = 'CREATE TABLE "meeting_reminder" ("ID" INTEGER UNIQUE, "chat_id" INTEGER, "meet_time" TEXT, "meet_location" TEXT, "meet_name" TEXT, "meet_schedule" TEXT, "meet_text_msg" TEXT, PRIMARY KEY ("ID"))'
            cursor.execute(query)
        except:
            pass
    except:
        pass
    
#--------------------------------------------------------------------    
def dop_meet_add(message):
    try:
        meet_time = message.text.split(';')[0].strip()
        meet_location = message.text.split(';')[1].strip()
        meet_name = message.text.split(';')[2].strip()
        meet_schedule = message.text.split(';')[3].strip()
        
        #Доп преобразования
        meet_time = '0'+ meet_time if len(meet_time.split(':')[0]) < 2 else meet_time 
        
        if meet_location.startswith('dion'):
            meet_location = meet_location.replace('dion','').strip()
            meet_location = f'https://dion.vc/event/{meet_location}'
            meet_name = f'Встреча "{meet_name}"'
        else:
            meet_name = f'Событие "{meet_name}"'
            
        
        with sqlite3.connect('meeting_reminder.db') as con:
            cursor = con.cursor()
            cursor.execute('INSERT INTO meeting_reminder (chat_id, meet_time, meet_location, meet_name, meet_schedule, meet_text_msg) VALUES (?, ?, ?, ?, ?, ?)',
                           (message.chat.id, meet_time, meet_location, meet_name, meet_schedule, message.text.strip()))
            con.commit()
        text = 'Встреча добавлена в "календарь"'
        bot.send_message(message.chat.id, text, reply_markup=keyboard_start)
    
    except:
        text = 'Строка имеет неверный формат'
        bot.send_message(message.chat.id, text, reply_markup=keyboard_start)
        return None

#--------------------------------------------------------------------
# просто функция, которая делает нам красивые строки для отправки пользователю
def get_meets_string(tasks):
    try:
        tasks_str = []
        for val in list(enumerate(tasks)): 
            if (val[1][3] == 'every day') \
                or (val[1][3] == 'every day on weekdays' and datetime.today().isocalendar().weekday in (1,2,3,4,5)) \
                or (val[1][3] == 'every monday' and datetime.today().isocalendar().weekday == 1) \
                or (val[1][3] == 'every tuesday' and datetime.today().isocalendar().weekday == 2) \
                or (val[1][3] == 'every wednesday' and datetime.today().isocalendar().weekday == 3) \
                or (val[1][3] == 'every thursday' and datetime.today().isocalendar().weekday == 4) \
                or (val[1][3] == 'every friday' and datetime.today().isocalendar().weekday == 5) \
                or (val[1][3] == 'every saturday' and datetime.today().isocalendar().weekday == 6) \
                or (val[1][3] == 'every sunday' and datetime.today().isocalendar().weekday == 7):
            
                tasks_str.append(f'{val[1][0]};{val[1][1]};{val[1][2]};{val[1][3]}\n')
            elif not(val[1][3].startswith('every')):
                try:
                    if date.fromisoformat(val[1][3]) == date.today():
                        tasks_str.append(f'{val[1][0]} {val[1][1]} {val[1][2]} {val[1][3]}\n')
                except:
                    print('Error get_meets_stringtry fromisoformat')
                    pass
    except:
        print('Error get_meets_string')
        pass
        
    return ''.join(tasks_str)    
                
                
def dop_meet_show(message):
    with sqlite3.connect('meeting_reminder.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT meet_time, meet_location, meet_name, meet_schedule, meet_text_msg FROM meeting_reminder WHERE chat_id=={}'.format(message.chat.id))
        tasks = get_meets_string(cursor.fetchall())
    bot.send_message(message.chat.id, tasks, reply_markup=keyboard_start)

    
#--------------------------------------------------------------------    
# выделяет одну встречу, которую пользователь хочет удалить
def dop_meet_del(message):
    keyboard_meet_delete = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    with sqlite3.connect('meeting_reminder.db') as con:
        cursor = con.cursor()
        # достаем все встречи 
        cursor.execute("SELECT cast(ID as TEXT) || ') ' || meet_text_msg FROM meeting_reminder WHERE chat_id=={}".format(message.chat.id))
        # достанем результат запроса
        tasks = cursor.fetchall()
        for value in tasks:
            keyboard_meet_delete.add(types.KeyboardButton(value[0]))
            
        msg = bot.send_message(message.chat.id, 'Выбери одну встречу из списка', reply_markup=keyboard_meet_delete)
                                 
        bot.register_next_step_handler(msg, dop_meet_del_)

# удаляет одну встречу
def dop_meet_del_(message):
    try:
        msg_text = int(message.text.split(') ')[0])
        with sqlite3.connect('meeting_reminder.db') as con:
            cursor = con.cursor()
            cursor.execute('DELETE FROM meeting_reminder WHERE chat_id==? AND ID==?', (message.chat.id, msg_text))
            con.commit()
            bot.send_message(message.chat.id, 'Ура, минус одна встреча!', reply_markup=keyboard_start)
    except:
        print('Error dop_meet_del_')
        pass

In [5]:
import sqlite3
import telebot
from telebot import types
from datetime import datetime, date
from time import sleep
from bot_token import token
# from dop_func_bot import create_db,dop_meet_add,get_meets_string,dop_meet_show,dop_meet_del,dop_meet_del_


bot = telebot.TeleBot(token)

# bot = telebot.TeleBot("TOKEN") # нужен токен вашего бота

# def handler(event,context): # основная функция для обработки действий бота
#     body = json.loads(event['body'])
#     update = telebot.types.Update.de_json(body)
#     bot.process_new_updates([update])
    

button_meet_add = types.KeyboardButton('Добавить встречу')
button_meet_show = types.KeyboardButton('Показать встречи на сегодня')
button_meet_delete = types.KeyboardButton('Удалить встречу')

# делаем клавиатуру
keyboard_start = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
keyboard_start.add(button_meet_add, button_meet_show)
keyboard_start.add(button_meet_delete)


# обрабатываем /start
@bot.message_handler(commands=['start'])
def start(message):
    create_db()
    start_text = "Привет! Я просто планирую встречи. Выбери действие"
    bot.send_message(message.chat.id, start_text, reply_markup=keyboard_start)

# добавление встречи
@bot.message_handler(regexp="Добавить встречу")
def meet_add(message):
    msg_reply = bot.send_message(message.chat.id, 'Давай добавим встречу! Напиши её параметры в чат в формате: (нужно ответить на это сообщение)')
    msg = bot.send_message(message.chat.id, '9:00;ссылка на встречу;название встречи;every day on weekdays - каждый будний день\n16:00;dion edoroshenko;Daily;every thursday - каждый четверг\n10:00;home;Что-то важное;every day - каждый день\n13:30;dion edoroshenko;Daily;YYYY-MM-DD - на конкретный день')
    
    bot.register_next_step_handler(msg_reply, dop_meet_add)

# показать все встречи
@bot.message_handler(regexp="Показать встречи на сегодня")
def meet_show(message):
    try:
        dop_meet_show(message)
    except:
        text = 'Здесь пусто. Не запланировано ниодной встречи'
        bot.send_message(message.chat.id, text, reply_markup=keyboard_start)

    bot.send_message(message.chat.id, 'Вот все встречи на сегодня', reply_markup=keyboard_start)

# Удалить встречу
@bot.message_handler(regexp="Удалить встречу")
def meet_del(message):
#     print('9--------')
    try:
        dop_meet_del(message)
    except:
        text = 'Что-то пошло не так. Попробуй снова'
        bot.send_message(message.chat.id, text, reply_markup=keyboard_start)

            
# обрабатываем другие сообщения
@bot.message_handler(content_types=["text"])
def repeat_all_messages(message):
    bot.send_message(message.chat.id, "Я тебя не понимаю" , reply_markup=keyboard_start)
    



In [6]:
bot.polling(none_stop=True)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))